In [110]:
import sqlite3
import json
import pandas as pd
import os
import textwrap
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from openai import OpenAI

In [111]:
tables_json_path_2 = r"C:\Users\coffe\OneDrive\Desktop\CITS5553 Capstone Project\spider_data\spider_data\train_spider.json"
if os.path.exists(tables_json_path_2):
    with open(tables_json_path_2, 'r', encoding='utf-8') as f:
        train_data = json.load(f)
    print(f"Number of tables/schemas: {len(train_data)}")
    if train_data :
        first_entry = train_data[5] 
        print(f"Keys: {list(first_entry.keys()) if isinstance(first_entry, dict) else 'Not a dictionary'}")
else:
    print("error.")

Number of tables/schemas: 7000
Keys: ['db_id', 'query', 'query_toks', 'query_toks_no_value', 'question', 'question_toks', 'sql']


In [112]:
dbid_question_list = [
    {'db_id': entry.get('db_id'), 'question': entry.get('question'), 'query': entry.get('query')}
    for entry in train_data if isinstance(entry, dict)
]
print(dbid_question_list[:5]) 

[{'db_id': 'department_management', 'question': 'How many heads of the departments are older than 56 ?', 'query': 'SELECT count(*) FROM head WHERE age  >  56'}, {'db_id': 'department_management', 'question': 'List the name, born state and age of the heads of departments ordered by age.', 'query': 'SELECT name ,  born_state ,  age FROM head ORDER BY age'}, {'db_id': 'department_management', 'question': 'List the creation year, name and budget of each department.', 'query': 'SELECT creation ,  name ,  budget_in_billions FROM department'}, {'db_id': 'department_management', 'question': 'What are the maximum and minimum budget of the departments?', 'query': 'SELECT max(budget_in_billions) ,  min(budget_in_billions) FROM department'}, {'db_id': 'department_management', 'question': 'What is the average number of employees of the departments whose rank is between 10 and 15?', 'query': 'SELECT avg(num_employees) FROM department WHERE ranking BETWEEN 10 AND 15'}]


In [116]:
every_x_th = dbid_question_list[::20]
# checking how many in total to be tested
len(every_x_th) 

350

In [117]:
for item in every_x_th:
    print(item['question'])
    print(item['db_id'])
    print(item['query'])

How many heads of the departments are older than 56 ?
department_management
SELECT count(*) FROM head WHERE age  >  56
What are the hosts of competitions whose theme is not "Aliens"?
farm
SELECT Hosts FROM farm_competition WHERE Theme !=  'Aliens'
Please show the themes of competitions with host cities having populations larger than 1000.
farm
SELECT T2.Theme FROM city AS T1 JOIN farm_competition AS T2 ON T1.City_ID  =  T2.Host_city_ID WHERE T1.Population  >  1000
what are the first name and last name of all candidates?
student_assessment
SELECT T2.first_name ,  T2.last_name FROM candidates AS T1 JOIN people AS T2 ON T1.candidate_id = T2.person_id
For every student who is registered for some course, how many courses are they registered for?
student_assessment
SELECT T1.student_id ,  count(*) FROM students AS T1 JOIN student_course_registrations AS T2 ON T1.student_id = T2.student_id GROUP BY T1.student_id
What are the ids of the students who either registered or attended a course?
stud

In [ ]:
from pathlib import Path
import subprocess, sys


PY = sys.executable


project_root = Path.cwd().parent  
print("CWD:", project_root)

answers = []

for item in every_x_th:
    result = subprocess.run(
        [PY, "-m", "src.agents.agent_a", "--query", item['question'], "--mode", "light", "--quiet"],
        cwd=project_root,                 
        text=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT         
    )
    answers.append(result.stdout.strip())
    



CWD: c:\Users\coffe\OneDrive\Desktop\CITS5553 Capstone Project\Project_Git\explainable-nl-query-db-agents


In [119]:
db_ground_truth = [item['db_id'] for item in every_x_th]
db_prediction = [ans.strip().strip('"') for ans in answers]

In [120]:
comparison_results = [gt == pred for gt, pred in zip(db_ground_truth, db_prediction)]
print(comparison_results)

[True, True, True, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, True, False, False, False, True, False, True, True, False, False, False, True, False, True, True, True, True, True, True, False, False, False, False, False, True, True, True, False, False, True, True, False, False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, False, False, True, True, True, False, True, True, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, True, True, False, True, True, True, True, True, True, True, True, True, False, True, True, True, True, True, True, False, True, True, False, True, True, True, True, False, True, True, False, True, True, True, True, True, True, True, True, True, True, True, False, True, False, False, False, True, True, True, True, True, True, True, True, True, False,

In [121]:
sum(comparison_results) / len (comparison_results)

0.7285714285714285

In [122]:
false_indices = [i for i, val in enumerate(comparison_results) if not val]
for i in false_indices:
    question = every_x_th[i]['question']
    ground_truth = db_ground_truth[i]
    prediction = db_prediction[i]
    print(f"Q: {question}\nTruth: {ground_truth} | Pred: {prediction}\n{'-'*60}")

Q: what are the first name and last name of all candidates?
Truth: student_assessment | Pred: candidate_poll
------------------------------------------------------------
Q: For every student who is registered for some course, how many courses are they registered for?
Truth: student_assessment | Pred: e_learning
------------------------------------------------------------
Q: What is the id and salary of the employee named Mark Young?
Truth: flight_1 | Pred: hr_1
------------------------------------------------------------
Q: What is the total number of students?
Truth: allergy_1 | Pred: e_learning
------------------------------------------------------------
Q: Who advises student 1004?
Truth: allergy_1 | Pred: college_2
------------------------------------------------------------
Q: How many students does each advisor have?
Truth: allergy_1 | Pred: college_2
------------------------------------------------------------
Q: What are the names of the top 8 countries by total invoice size an

In [123]:
correct_pairs = [v for v, f in zip(every_x_th, comparison_results) if f]

In [124]:
len(correct_pairs)

255

In [125]:
for item in correct_pairs:
    print(item['question'])
    print(item['db_id'])
    print(item['query'])

How many heads of the departments are older than 56 ?
department_management
SELECT count(*) FROM head WHERE age  >  56
What are the hosts of competitions whose theme is not "Aliens"?
farm
SELECT Hosts FROM farm_competition WHERE Theme !=  'Aliens'
Please show the themes of competitions with host cities having populations larger than 1000.
farm
SELECT T2.Theme FROM city AS T1 JOIN farm_competition AS T2 ON T1.City_ID  =  T2.Host_city_ID WHERE T1.Population  >  1000
What are the ids of the students who either registered or attended a course?
student_assessment
SELECT student_id FROM student_course_registrations UNION SELECT student_id FROM student_course_attendance
What is the id of the shortest trip?
bike_1
SELECT id FROM trip ORDER BY duration LIMIT 1
What is the latitude, longitude, and city of the station from which the trip with smallest duration started?
bike_1
SELECT T1.lat ,  T1.long ,  T1.city FROM station AS T1 JOIN trip AS T2 ON T1.id  =  T2.start_station_id ORDER BY T2.durati

In [126]:
correct_db_prediction = [v for v, f in zip(db_prediction, comparison_results) if f]

In [127]:
correct_db_prediction

['department_management',
 'farm',
 'farm',
 'student_assessment',
 'bike_1',
 'bike_1',
 'bike_1',
 'bike_1',
 'bike_1',
 'book_2',
 'musical',
 'musical',
 'twitter_1',
 'twitter_1',
 'product_catalog',
 'product_catalog',
 'flight_1',
 'flight_1',
 'flight_1',
 'allergy_1',
 'allergy_1',
 'store_1',
 'store_1',
 'journal_committee',
 'customers_card_transactions',
 'customers_card_transactions',
 'customers_card_transactions',
 'race_track',
 'race_track',
 'coffee_shop',
 'insurance_fnol',
 'medicine_enzyme_interaction',
 'medicine_enzyme_interaction',
 'university_basketball',
 'phone_1',
 'climbing',
 'climbing',
 'body_builder',
 'election_representative',
 'apartment_rentals',
 'apartment_rentals',
 'apartment_rentals',
 'apartment_rentals',
 'game_injury',
 'soccer_1',
 'performance_attendance',
 'college_2',
 'college_2',
 'college_2',
 'college_2',
 'college_2',
 'debate',
 'insurance_and_eClaims',
 'insurance_and_eClaims',
 'customers_and_invoices',
 'customers_and_invoices

In [128]:
PY = sys.executable


project_root = Path.cwd().parent  
print("CWD:", project_root)

answers_agent_b = []

for item in correct_pairs:
    result = subprocess.run(
        [PY, "-m", "src.agents.agent_b", "--query", item['question'], "--database", item['db_id'], "--mode", "medium", "--quiet"],
        cwd=project_root,                 
        text=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT          
    )
    answers_agent_b.append(result.stdout.strip())
    

CWD: c:\Users\coffe\OneDrive\Desktop\CITS5553 Capstone Project\Project_Git\explainable-nl-query-db-agents


In [129]:
parsed_answers = []

for i in answers_agent_b:
    try:
        data = json.loads(i)   # convert string → dict
        parsed_answers.append(data)
    except json.JSONDecodeError as e:
        print(" Could not parse:", i)
        print("Error:", e)


for ans in parsed_answers:
    print("Query:", ans["User Query"])
    print("Database name:", ans["Database Name"])
    print("Tables:", ans["Tables"])
    print()

Query: How many heads of the departments are older than 56 ?
Database name: department_management
Tables: ['head', 'management']

Query: What are the hosts of competitions whose theme is not "Aliens"?
Database name: farm
Tables: ['farm competition', 'city']

Query: Please show the themes of competitions with host cities having populations larger than 1000.
Database name: farm
Tables: ['farm competition', 'city']

Query: What are the ids of the students who either registered or attended a course?
Database name: student_assessment
Tables: ['student course registrations', 'student course attendance', 'students']

Query: What is the id of the shortest trip?
Database name: bike_1
Tables: ['trip']

Query: What is the latitude, longitude, and city of the station from which the trip with smallest duration started?
Database name: bike_1
Tables: ['trip', 'station']

Query: Which days had a minimum dew point smaller than any day in zip code 94107, and in which zip codes were those measurements ta

In [130]:
PY = sys.executable

# 2) Point cwd to your project root (adjust if your notebook isn't in project_root/notebooks/)
project_root = Path.cwd().parent  # e.g., notebooks/ -> project root
print("CWD:", project_root)

answers_agent_c = []

for item in parsed_answers:
    result = subprocess.run(
        [PY, "-m", "src.agents.agent_c", "--query", item["User Query"], "--database", item['Database Name'], "--tables", json.dumps(item['Tables']), "--quiet"],
        cwd=project_root,                 # critical so -m finds src.agents.agent_c
        text=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT          # merge stderr so you see errors/logs
    )
    answers_agent_c.append(result.stdout.strip())

CWD: c:\Users\coffe\OneDrive\Desktop\CITS5553 Capstone Project\Project_Git\explainable-nl-query-db-agents


In [131]:
answers_agent_c = [q.replace("\n", " ") for q in answers_agent_c]

In [132]:
answers_agent_c

['SELECT COUNT(*) FROM head WHERE age > 56;',
 "SELECT DISTINCT c.official_name FROM farm_competition fc JOIN city c ON fc.host_city_id = c.city_id WHERE fc.theme <> 'Aliens'",
 'SELECT fc.theme FROM farm_competition fc JOIN city c ON fc.host_city_id = c.city_id WHERE c.population > 1000',
 'SELECT DISTINCT student_id FROM Student_Course_Registrations UNION SELECT DISTINCT student_id FROM Student_Course_Attendance;',
 'SELECT id FROM trip ORDER BY duration ASC LIMIT 1',
 'SELECT s.lat, s.long, s.city FROM station s JOIN trip t ON s.id = t.start_station_id ORDER BY t.duration ASC LIMIT 1',
 "SELECT w.zip_code  FROM weather w  WHERE w.min_dew_point_f < (SELECT MIN(w2.min_dew_point_f) FROM weather w2 WHERE w2.zip_code = '94107')",
 'SELECT start_station_id , start_station_name FROM trip GROUP BY start_station_id , start_station_name HAVING COUNT(id) >= 200',
 'SELECT date FROM weather WHERE mean_sea_level_pressure_inches BETWEEN 30.3 AND 31;',
 'SELECT Publisher FROM publication WHERE Pri

In [133]:
SQL_truth = []
Query_truth = []

for item in correct_pairs:
    print(item['question'])
    print(item['query'])
    SQL_truth.append(item['query'])
    Query_truth.append(item['question'])

How many heads of the departments are older than 56 ?
SELECT count(*) FROM head WHERE age  >  56
What are the hosts of competitions whose theme is not "Aliens"?
SELECT Hosts FROM farm_competition WHERE Theme !=  'Aliens'
Please show the themes of competitions with host cities having populations larger than 1000.
SELECT T2.Theme FROM city AS T1 JOIN farm_competition AS T2 ON T1.City_ID  =  T2.Host_city_ID WHERE T1.Population  >  1000
What are the ids of the students who either registered or attended a course?
SELECT student_id FROM student_course_registrations UNION SELECT student_id FROM student_course_attendance
What is the id of the shortest trip?
SELECT id FROM trip ORDER BY duration LIMIT 1
What is the latitude, longitude, and city of the station from which the trip with smallest duration started?
SELECT T1.lat ,  T1.long ,  T1.city FROM station AS T1 JOIN trip AS T2 ON T1.id  =  T2.start_station_id ORDER BY T2.duration LIMIT 1
Which days had a minimum dew point smaller than any da

In [134]:
print(answers_agent_c[1])
print(SQL_truth[1])

SELECT DISTINCT c.official_name FROM farm_competition fc JOIN city c ON fc.host_city_id = c.city_id WHERE fc.theme <> 'Aliens'
SELECT Hosts FROM farm_competition WHERE Theme !=  'Aliens'


In [135]:
embeddings = OpenAIEmbeddings()  
vectorstore_pred = FAISS.from_texts(answers_agent_c, embeddings)
vectorstore_truth = FAISS.from_texts(SQL_truth, embeddings)

In [136]:
import numpy as np

# Extract embeddings directly from FAISS indices
pred_embeddings = vectorstore_pred.index.reconstruct_n(0, vectorstore_pred.index.ntotal)
truth_embeddings = vectorstore_truth.index.reconstruct_n(0, vectorstore_truth.index.ntotal)

pred_embeddings = np.array(pred_embeddings)
truth_embeddings = np.array(truth_embeddings)

print(pred_embeddings.shape, truth_embeddings.shape)

(255, 1536) (255, 1536)


In [137]:
from numpy.linalg import norm

pred_norm = pred_embeddings / norm(pred_embeddings, axis=1, keepdims=True)
truth_norm = truth_embeddings / norm(truth_embeddings, axis=1, keepdims=True)

In [138]:
cos_sims = np.sum(pred_norm * truth_norm, axis=1)
similarity_score = []
for i, score in enumerate(cos_sims):
    print(f"Query {i}: cosine similarity = {score:.3f}")
    similarity_score.append(score)

Query 0: cosine similarity = 0.974
Query 1: cosine similarity = 0.909
Query 2: cosine similarity = 0.943
Query 3: cosine similarity = 0.973
Query 4: cosine similarity = 0.997
Query 5: cosine similarity = 0.964
Query 6: cosine similarity = 0.969
Query 7: cosine similarity = 0.989
Query 8: cosine similarity = 0.984
Query 9: cosine similarity = 0.951
Query 10: cosine similarity = 0.981
Query 11: cosine similarity = 0.951
Query 12: cosine similarity = 0.963
Query 13: cosine similarity = 0.980
Query 14: cosine similarity = 0.925
Query 15: cosine similarity = 0.953
Query 16: cosine similarity = 0.983
Query 17: cosine similarity = 0.928
Query 18: cosine similarity = 0.974
Query 19: cosine similarity = 0.920
Query 20: cosine similarity = 0.937
Query 21: cosine similarity = 0.970
Query 22: cosine similarity = 0.972
Query 23: cosine similarity = 0.944
Query 24: cosine similarity = 0.990
Query 25: cosine similarity = 0.987
Query 26: cosine similarity = 0.948
Query 27: cosine similarity = 0.986
Qu

In [139]:
def to_serializable(obj):
    if isinstance(obj, (np.generic,)):   # catches np.float32, np.int64, etc.
        return obj.item()
    return str(obj)   # fallback: turn into string

final_comparison = [
    {"query": q, "sql_truth": gt, "sql_pred": pred , "similarity": sim}
    for q, gt, pred, sim in zip(Query_truth, SQL_truth, answers_agent_c, similarity_score)
]

# pretty print
print(json.dumps(final_comparison, indent=2,default=to_serializable))

[
  {
    "query": "How many heads of the departments are older than 56 ?",
    "sql_truth": "SELECT count(*) FROM head WHERE age  >  56",
    "sql_pred": "SELECT COUNT(*) FROM head WHERE age > 56;",
    "similarity": 0.9744021892547607
  },
  {
    "query": "What are the hosts of competitions whose theme is not \"Aliens\"?",
    "sql_truth": "SELECT Hosts FROM farm_competition WHERE Theme !=  'Aliens'",
    "sql_pred": "SELECT DISTINCT c.official_name FROM farm_competition fc JOIN city c ON fc.host_city_id = c.city_id WHERE fc.theme <> 'Aliens'",
    "similarity": 0.9094228148460388
  },
  {
    "query": "Please show the themes of competitions with host cities having populations larger than 1000.",
    "sql_truth": "SELECT T2.Theme FROM city AS T1 JOIN farm_competition AS T2 ON T1.City_ID  =  T2.Host_city_ID WHERE T1.Population  >  1000",
    "sql_pred": "SELECT fc.theme FROM farm_competition fc JOIN city c ON fc.host_city_id = c.city_id WHERE c.population > 1000",
    "similarity": 0